In [19]:
# Imports and Setup
%load_ext autoreload
%autoreload 2

from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
import json
from sentence_transformers import SentenceTransformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Load concepts
with open('concepts.json') as f:
    concepts = json.load(f)

# Print first 5 adjectives
print(f'These concept sets are available: {list(concepts.keys())}')

These concept sets are available: ['anthropic', 'adjectives']


In [23]:
concept_name = 'adjectives'

### Nomic model

In [14]:
# # # Initialize SentenceTransformer model
# model_name = 'nomic'
# model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

<All keys matched successfully>


In [ ]:
# # Generate embeddings
# queries = concepts['adjectives']
# query_embeddings = model.encode(queries)
# print(f'shape of query_embeddings: {query_embeddings.shape}')

### NV-embed-v2

In [11]:
# from transformers import AutoModel

# # load model with tokenizer
# model_name = 'nv-embed-v2'
# model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True)

In [12]:
# import torch
# import torch.nn.functional as F
# from transformers import AutoTokenizer, AutoModel

# # Each query needs to be accompanied by an corresponding instruction describing the task.
# task_name_to_instruct = {"example": "Given a query word, retrieve semantically similar words.",}

# query_prefix = "Instruct: "+task_name_to_instruct["example"]+"\nQuery: "
# queries =

# # No instruction needed for retrieval passages
# passage_prefix = ""
# passages = [
#     "Since you're reading this, you are probably someone from a judo background or someone who is just wondering how judo techniques can be applied under wrestling rules. So without further ado, let's get to the question. Are Judo throws allowed in wrestling? Yes, judo throws are allowed in freestyle and folkstyle wrestling. You only need to be careful to follow the slam rules when executing judo throws. In wrestling, a slam is lifting and returning an opponent to the mat with unnecessary force.",
#     "Below are the basic steps to becoming a radiologic technologist in Michigan:Earn a high school diploma. As with most careers in health care, a high school education is the first step to finding entry-level employment. Taking classes in math and science, such as anatomy, biology, chemistry, physiology, and physics, can help prepare students for their college studies and future careers.Earn an associate degree. Entry-level radiologic positions typically require at least an Associate of Applied Science. Before enrolling in one of these degree programs, students should make sure it has been properly accredited by the Joint Review Committee on Education in Radiologic Technology (JRCERT).Get licensed or certified in the state of Michigan."
# ]



# # get the embeddings
# max_length = 32768
# query_embeddings = model.encode(queries, instruction=query_prefix, max_length=max_length)
# passage_embeddings = model.encode(passages, instruction=passage_prefix, max_length=max_length)

# # normalize embeddings
# query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
# passage_embeddings = F.normalize(passage_embeddings, p=2, dim=1)

# # get the embeddings with DataLoader (spliting the datasets into multiple mini-batches)
# # batch_size=2
# # query_embeddings = model._do_encode(queries, batch_size=batch_size, instruction=query_prefix, max_length=max_length, num_workers=32, return_numpy=True)
# # passage_embeddings = model._do_encode(passages, batch_size=batch_size, instruction=passage_prefix, max_length=max_length, num_workers=32, return_numpy=True)

# scores = (query_embeddings @ passage_embeddings.T) * 100
# print(scores.tolist())
# # [[87.42693328857422, 0.46283677220344543], [0.965264618396759, 86.03721618652344]]

### Gemma-2-9B based

In [35]:
from sentence_transformers import SentenceTransformer
import torch

# Load the model, optionally in float16 precision for faster inference
embedding_name = 'BAAI/bge-multilingual-gemma2'
model = SentenceTransformer(embedding_name, model_kwargs={"torch_dtype": torch.float16, 'device_map': 'cuda'})

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.91s/it]


In [25]:
# Prepare a prompt given an instruction
instruction = 'Given a web search query, retrieve relevant passages that answer the query.'
# instruction = 'Given a query word, retrieve semantically similar target words.'
prompt = f'<instruct>{instruction}\n<query>'

# Compute the query and document embeddings
concept_embeddings = model.encode(concepts[concept_name], prompt=prompt)

In [34]:
# Compute the cosine similarity between the query and document embeddings
fn_name = model.similarity_fn_name
print(f'similarity fn: {fn_name}')

similarities = model.similarity(concept_embeddings, concept_embeddings)
print(similarities)



similarity fn: cosine
tensor([[1.0000, 0.3770, 0.3789,  ..., 0.2612, 0.2250, 0.3523],
        [0.3770, 1.0000, 0.4480,  ..., 0.3066, 0.3049, 0.2629],
        [0.3789, 0.4480, 1.0010,  ..., 0.2615, 0.2410, 0.2573],
        ...,
        [0.2612, 0.3066, 0.2615,  ..., 0.9990, 0.4751, 0.3674],
        [0.2250, 0.3049, 0.2410,  ..., 0.4751, 0.9995, 0.4421],
        [0.3523, 0.2629, 0.2573,  ..., 0.3674, 0.4421, 0.9995]],
       dtype=torch.float16)


In [29]:
import pandas as pd
df_sim = pd.DataFrame(similarities, index=concepts[concept_name], columns=concepts[concept_name])

In [33]:
word = 'warm'

pd.set_option('display.max_rows', 100)
df_sim[word].sort_values(ascending=False).head(100)

warm           0.999512
hot            0.533203
cold           0.507324
calm           0.468994
cool           0.461670
weak           0.444092
cozy           0.429199
wet            0.427979
wary           0.426758
warlike        0.425049
wide           0.425049
soft           0.422119
strong         0.420166
wild           0.415771
firm           0.412842
sweet          0.405518
round          0.397949
friendly       0.396240
happy          0.395752
rich           0.394287
fresh          0.393066
wise           0.392578
dark           0.391113
bright         0.389648
familiar       0.389404
smart          0.389160
wealthy        0.387695
smooth         0.384033
sharp          0.383789
normal         0.383057
comfortable    0.382812
romantic       0.382812
dry            0.381348
spicy          0.376709
brown          0.375488
safe           0.374023
narrow         0.373535
charming       0.370117
lovely         0.370117
brave          0.369385
tight          0.369141
damp           0

In [41]:
import pickle as pkl

output = {
    'concepts': {concept_name: concepts[concept_name]},
    'similarities': similarities,
    'config': {
        'concept_name': concept_name,
        'embedding_name': embedding_name,
        'fn_name': fn_name,
    }
}

fname = f'{concept_name}_{embedding_name}_{fn_name}.pkl'
fname = fname.replace('/', '-')
with open(fname, 'wb') as f:
    pkl.dump(output, f)

In [43]:
output['concepts'].keys()

dict_keys(['adjectives'])